## Next.js middleware

Middleware allows you to run code before a request is completed. Then, based on the incoming request, you can modify the response by rewriting, redirecting, modifying the request or response headers, or responding directly.

Middleware runs before cached content and routes are matched.

To use built-in middleware we first have to create a file in "src" folder (or root folder if we don't have src folder in our project), and set up the matcher.

Matcher allows you to filter Middleware to run (or not run like in the example) on specific paths. If you don't add any matcher, Middleware will be invoked for every route in your project.

In [ ]:
## middleware.js

import NextAuth from "next-auth";
import { authConfig} from "./lib/auth.config"

export default NextAuth(authConfig).auth;


export const config = {
    matcher: [
      '/((?!api|_next/static|_next/image|favicon.ico|sitemap.xml|robots.txt).*)',
    ],
  }

## below one is without imports, was not working but I keep it just in case (it's a solution from tutorial)
# export const config = {
#     matcher: ['/((?!api|static|.*\\..*|_next).*'],
# }

Now we have to add new callback function in the auth.js file, where we can define our authorization rules. The problem is that we are using node js libraries (for example bcrypt, mongoose), but middleware is independent from any dependent node library. To prevent this we have first to create a new file in app/lib folder.

In [ ]:
## auth.config.js

export const authConfig = {
    pages: {
        signIn: "/login"  ## this is a page the use will be redirected to, when he's not logged in (when the callback func...
        ## ... will return false 
    },
    providers: [],
    callbacks: {
        ## when you log in via OAuth you have more information about the user, but if you use credentials you only have
        ## user email. If you want to "get" more information, especially when you want check if the user is admin you have to
        ## use asynchronous functions jwt() and session()
        ## more info at: https://next-auth.js.org/configuration/callbacks
        async jwt({token, user}) {
            if(user) {
                token.id = user.id;
                token.isAdmin = user.isAdmin;
            }
            return token
        },
        async session({session, token}) {
            if (token) {
                session.user.id = token.id;
                session.user.isAdmin = token.isAdmin;
            }
            return session
        },
        authorized({ auth, request }) {
            
        const user = auth?.user; # save the authorized user as a value of variable user
            
        const isOnAdminPanel = request.nextUrl?.pathname.startsWith("/admin");
        const isOnBlogPage = request.nextUrl?.pathname.startsWith("/blog");
        const isOnLoginPage = request.nextUrl?.pathname.startsWith("/login");
  
        ## ONLY ADMIN CAN REACH THE ADMIN DASHBOARD
  
        if (isOnAdminPanel && !user?.isAdmin) {
          return false;
        }
  
        ## ONLY AUTHENTICATED USERS CAN REACH THE BLOG PAGE
  
        if (isOnBlogPage && !user) {
          return false;
        }
  
        ## ONLY UNAUTHENTICATED USERS CAN REACH THE LOGIN PAGE
  
        if (isOnLoginPage && user) {
          return Response.redirect(new URL("/", request.nextUrl));
        }
  
        return true
      },
    },
  };

Now, the auth.js must be modified.

In [ ]:
import NextAuth from "next-auth";
import GitHub from "next-auth/providers/github";
import Credentials from "next-auth/providers/credentials";
import { connectToDb } from "./utils";
import { User } from "./models";
import bcrypt from "bcryptjs";
import {authConfig} from "./auth.config" ## import authConfig

const login = async (credentials) => {
  try {
    connectToDb();
    const user = await User.findOne({username: credentials.username})

    if (!user) {
      throw new Error("Wrong credentials!");
    }

    const isPasswordCorrect = await bcrypt.compare(credentials.password, user.password);

    if (!isPasswordCorrect) {
      throw new Error("Wrong credentials")
    }

    return user

  } catch (error) {
    console.log(error)
    throw new Error("Failed to login")
  }
}

export const { handlers, signIn, signOut, auth } = NextAuth({
  ...authConfig,  ## spread authConfig
  providers: [
    GitHub({
      clientId: process.env.AUTH_GITHUB_ID,
      clientSecret: process.env.AUTH_GITHUB_SECRET,
    }),
    Credentials({
      async authorize(credentials) {
        try {
          const user = await login(credentials);
          return user
        } catch (error) {
          return null
        }
      }
    })
  ],
  callbacks: {
    async signIn({user, account, profile}) {
      if (account?.provider === "github") {
        connectToDb();
        try {
          const user = await User.findOne({email: profile?.email});
          if (!user) {
            const newUser = new User({
              username: profile?.login,
              email: profile?.email,
              img: profile?.avatar_url,
            });

            await newUser.save();
          }

        } catch (error) {
          console.log(error)
          return false
        }
      }
      return true
    },
    ...authConfig.callbacks  ## spread callbacks from auth.config.js
  },
})